#Lib Import

In [21]:
# NLTK
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
nltk.download('stopwords')

#Spacy
import spacy
nlp = spacy.load('en')

# Other
import re
import json
import string
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder

#Keras
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#Data loading and cleaning

In [22]:
#load data
import pandas as pd
reviews_train = pd.read_csv("https://drive.google.com/uc?export=download&id=1InpAOdyHowB805CFVEWg-HHiXXh0lpBS").astype(str)

#show first 5 records
reviews_train.head()

,review,aspect,sentiment
0,It has be my TD branch for the past 15 years. ...,staff,positive
1,It has be my TD branch for the past 15 years. ...,service,positive
2,So rude castumer service,service,negative
3,Staff were very welcoming and friendly. I had ...,staff,positive
4,Here are 3 reasons TD is the worst bank for sm...,staff,negative


In [23]:
#data size
reviews_train.shape

(90, 3)

In [0]:
#fix typos
reviews_train.sentiment[reviews_train.sentiment == 'netrual'] = 'neutral' 
reviews_train.sentiment[reviews_train.sentiment == 'positive '] = 'positive' 
reviews_train.sentiment[reviews_train.sentiment == 'negative '] = 'negative' 
reviews_train.aspect[reviews_train.aspect == 'servicve'] = 'service' 
reviews_train.aspect[reviews_train.aspect == 'line '] = 'line'

In [0]:
#aspect name change
reviews_train.aspect[reviews_train.aspect == 'service'] = 'service quality' 
reviews_train.aspect[reviews_train.aspect == 'staff'] = 'staff friendliness'

In [26]:
#drop neutral records to reduce imbalance for now
reviews_train =reviews_train[reviews_train.sentiment != 'neutral'  ]
reviews_train.groupby('sentiment').size()

sentiment
negative    44
positive    40
dtype: int64

In [27]:
#fix typos
reviews_train.aspect[reviews_train.aspect == 'servicve'] = 'service' 
reviews_train.aspect[reviews_train.aspect == 'line '] = 'line'

#drop na column to reduce imbalance for now
reviews_train =reviews_train[reviews_train.aspect != 'na'  ]
reviews_train.groupby('aspect').size()

aspect
line                  10
service quality       24
staff friendliness    48
dtype: int64

In [28]:
#8 rows dropped in total 
reviews_train.shape

(82, 3)

#Aspect analyzer


In [0]:
#asepct classifier NN sturcture, the output layer is 3 neurons corresponding three aspects
aspect_model = Sequential()
aspect_model.add(Dense(512, input_shape=(3000,), activation='relu'))
aspect_model.add((Dense(256, activation='relu')))
aspect_model.add((Dense(128, activation='relu')))
aspect_model.add(Dense(3, activation='softmax'))
#compile model
aspect_model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [0]:
#asepct classifier NN sturcture, the output layer is 3 neurons corresponding three aspects
aspect_model = Sequential()
aspect_model.add(Dense(512, input_shape=(6000,), activation='relu'))
aspect_model.add((Dense(256, activation='relu')))
aspect_model.add((Dense(128, activation='relu')))
aspect_model.add(Dense(3, activation='softmax'))
#compile model
aspect_model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [0]:
#tokenize review text
vocab_size = 6000 # We set a maximum size for the vocabulary
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(reviews_train.review)
reviews_tokenized = pd.DataFrame(tokenizer.texts_to_matrix(reviews_train.review))

In [32]:
reviews_tokenized

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,5960,5961,5962,5963,5964,5965,5966,5967,5968,5969,5970,5971,5972,5973,5974,5975,5976,5977,5978,5979,5980,5981,5982,5983,5984,5985,5986,5987,5988,5989,5990,5991,5992,5993,5994,5995,5996,5997,5998,5999
0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
78,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
79,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
80,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
#encode aspect classes
label_encoder = LabelEncoder()
integer_category = label_encoder.fit_transform(reviews_train.aspect)
encoded_aspect = to_categorical(integer_category)

In [34]:
#fit aspect classifier
aspect_model.fit(reviews_tokenized, encoded_aspect, epochs=100, verbose=1)

Epoch 1/100
82/82 [==============================] - 1s 9ms/step - loss: 1.0398 - acc: 0.5976
Epoch 2/100
82/82 [==============================] - 0s 2ms/step - loss: 0.8921 - acc: 0.5854
Epoch 3/100
82/82 [==============================] - 0s 2ms/step - loss: 0.7471 - acc: 0.6341
Epoch 4/100
82/82 [==============================] - 0s 2ms/step - loss: 0.6478 - acc: 0.6707
Epoch 5/100
82/82 [==============================] - 0s 2ms/step - loss: 0.5659 - acc: 0.7073
Epoch 6/100
82/82 [==============================] - 0s 2ms/step - loss: 0.5004 - acc: 0.7439
Epoch 7/100
82/82 [==============================] - 0s 2ms/step - loss: 0.4537 - acc: 0.7683
Epoch 8/100
82/82 [==============================] - 0s 2ms/step - loss: 0.4325 - acc: 0.7439
Epoch 9/100
82/82 [==============================] - 0s 2ms/step - loss: 0.4165 - acc: 0.7561
Epoch 10/100
82/82 [==============================] - 0s 2ms/step - loss: 0.3975 - acc: 0.7683
Epoch 11/100
82/82 [==============================] - 0s 2m

#Sentiment analyzer


In [0]:
#sentiment classifier NN sturcture, the output layer is 2 neurons corresponding two sentiments
sentiment_model = Sequential()
sentiment_model.add(Dense(512, input_shape=(6000,), activation='relu'))
sentiment_model.add((Dense(256, activation='relu')))
sentiment_model.add((Dense(128, activation='relu')))
sentiment_model.add(Dense(2, activation='softmax'))
#compile model
sentiment_model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [0]:
#tokenize review text
vocab_size = 6000 # We set a maximum size for the vocabulary
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(reviews_train.review)
reviews_tokenized = pd.DataFrame(tokenizer.texts_to_matrix(reviews_train.review))

#encode sentiment classes
label_encoder_2 = LabelEncoder()
integer_sentiment = label_encoder_2.fit_transform(reviews_train.sentiment)
encoded_sentiment = to_categorical(integer_sentiment)

In [37]:
#fit aspect classifier
sentiment_model.fit(reviews_tokenized, encoded_sentiment, epochs=100, verbose=1)

Epoch 1/100
82/82 [==============================] - 1s 9ms/step - loss: 0.6712 - acc: 0.4878
Epoch 2/100
82/82 [==============================] - 0s 2ms/step - loss: 0.5140 - acc: 0.7439
Epoch 3/100
82/82 [==============================] - 0s 2ms/step - loss: 0.3660 - acc: 1.0000
Epoch 4/100
82/82 [==============================] - 0s 2ms/step - loss: 0.2209 - acc: 1.0000
Epoch 5/100
82/82 [==============================] - 0s 2ms/step - loss: 0.1086 - acc: 1.0000
Epoch 6/100
82/82 [==============================] - 0s 2ms/step - loss: 0.0475 - acc: 1.0000
Epoch 7/100
82/82 [==============================] - 0s 2ms/step - loss: 0.0200 - acc: 1.0000
Epoch 8/100
82/82 [==============================] - 0s 2ms/step - loss: 0.0059 - acc: 1.0000
Epoch 9/100
82/82 [==============================] - 0s 2ms/step - loss: 0.0021 - acc: 1.0000
Epoch 10/100
82/82 [==============================] - 0s 2ms/step - loss: 8.4613e-04 - acc: 1.0000
Epoch 11/100
82/82 [==============================] - 0

#Testing & Results

In [0]:
test_reviews = [
    "Good, fast service.",
    "The teller was very pleasant.",
    "The line is way too long and staffs are not so friendly",
    "Rude & unprofessional/inefficient staff (including Randy, the manager on site). Time to switch banks.",
    "The line here is way too long"
]


# Aspect preprocessing
test_reviews = [review.lower() for review in test_reviews]
test_aspect_terms = []

#chunking method for keywords extraction
for review in nlp.pipe(test_reviews):
    chunks = [(chunk.root.text) for chunk in review.noun_chunks if chunk.root.pos_ == 'NOUN']
    test_aspect_terms.append(' '.join(chunks))

test_aspect_terms = pd.DataFrame(tokenizer.texts_to_matrix(test_aspect_terms))

# Sentiment preprocessing
test_sentiment_terms = []
for review in nlp.pipe(test_reviews):
        if review.is_parsed:
            test_sentiment_terms.append(' '.join([token.lemma_ for token in review if (not token.is_stop and not token.is_punct and (token.pos_ == "ADJ" or token.pos_ == "VERB"))]))
        else:
            test_sentiment_terms.append('') 


test_sentiment_terms = pd.DataFrame(tokenizer.texts_to_matrix(test_sentiment_terms))


In [43]:
#get test results
test_aspect_categories = label_encoder.inverse_transform(aspect_model.predict_classes(test_aspect_terms))
test_sentiment = label_encoder_2.inverse_transform(sentiment_model.predict_classes(test_sentiment_terms))

#print results

print ("\n\nHere are the model outputs for 5 unseen branch reviews from Google:\n\n")
for i in range (len(test_reviews)):
    print("Review " + str(i+1) + ":" + test_reviews[i] )
    print("Review " + str(i+1) + " is expressing " + test_sentiment[i] +" opinion about" + test_aspect_categories[i]+'\n')



Here are the model outputs for 5 unseen branch reviews from Google:


Review 1:good, fast service.
Review 1 is expressing positive opinion aboutservice quality

Review 2:the teller was very pleasant.
Review 2 is expressing positive opinion aboutstaff friendliness

Review 3:the line is way too long and staffs are not so friendly
Review 3 is expressing positive opinion aboutline

Review 4:rude & unprofessional/inefficient staff (including randy, the manager on site). time to switch banks.
Review 4 is expressing negative opinion aboutstaff friendliness

Review 5:the line here is way too long
Review 5 is expressing negative opinion aboutline

